In [2]:
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

# Identificación en Kaggle
api = KaggleApi()
api.authenticate()

# Definir el dataset y la ubicación
dataset = 'sgpjesus/bank-account-fraud-dataset-neurips-2022'
file_name = 'Base.csv'
zip_file = 'bank-account-fraud-dataset-neurips-2022.zip'

# Descargar la dataset
api.dataset_download_files(dataset, path='.', unzip=False)

# Extraer el csv específico
with zipfile.ZipFile(zip_file, 'r') as z:
    z.extract(file_name, "../data/raw")

# Borrar archivo .zip
os.remove(zip_file)

Dataset URL: https://www.kaggle.com/datasets/sgpjesus/bank-account-fraud-dataset-neurips-2022


In [22]:
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Configurar la conexión
connection_string = f"postgresql://{os.getenv('PGUSER')}:{os.getenv('PGPASSWORD')}@{os.getenv('PGHOST')}/{os.getenv('PGNAME')}"
engine = create_engine(connection_string).execution_options(autocommit=True)

# Leer el CSV con pandas
bank_account_fraud_df = pd.read_csv("../data/raw/Base.csv")

# Crear la tabla base
create_table_query = '''
CREATE TABLE IF NOT EXISTS base (
    id SERIAL PRIMARY KEY,
    fraud_bool INTEGER,
    income FLOAT,
    name_email_similarity FLOAT,
    prev_address_months_count INTEGER,
    current_address_months_count INTEGER,
    customer_age INTEGER,
    days_since_request FLOAT,
    intended_balcon_amount FLOAT,
    payment_type VARCHAR(255),
    zip_count_4w INTEGER,
    velocity_6h FLOAT,
    velocity_24h FLOAT,
    velocity_4w FLOAT,
    bank_branch_count_8w INTEGER,
    date_of_birth_distinct_emails_4w INTEGER,
    employment_status VARCHAR(255),
    credit_risk_score INTEGER,
    email_is_free INTEGER,
    housing_status VARCHAR(255),
    phone_home_valid INTEGER,
    phone_mobile_valid INTEGER,
    bank_months_count INTEGER,
    has_other_cards INTEGER,
    proposed_credit_limit FLOAT,
    foreign_request INTEGER,
    source VARCHAR(255),
    session_length_in_minutes FLOAT,
    device_os VARCHAR(255),
    keep_alive_session INTEGER,
    device_distinct_emails_8w INTEGER,
    device_fraud_count INTEGER,
    month INTEGER
);
'''

with engine.connect() as connection:
    connection.execute(text(create_table_query))
    connection.rollback()  # Rollback total de la transacción pendiente

# Insertar los datos en la tabla
bank_account_fraud_df.to_sql('base', con=engine, if_exists='append', index=False)

# Crear la session de la Bases de Datos 
with Session(engine) as session:
    # Commit cambios
    session.commit()

# Cerrar session
session.close()

OperationalError: (psycopg2.OperationalError) connection to server at "roundhouse.proxy.rlwy.net" (35.212.97.29), port 36778 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [1]:
import pandas as pd

# Cargar el dataset en un dataframe de Pandas
bank_account_fraud_df = pd.read_csv(".\Base.csv")
bank_account_fraud_df.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,...,0,1500.0,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,...,0,1500.0,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,...,0,200.0,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,...,0,200.0,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,...,0,200.0,0,INTERNET,3.743048,other,0,1,0,0


In [2]:
bank_account_fraud_df["device_os"].unique()

array(['linux', 'other', 'windows', 'x11', 'macintosh'], dtype=object)

In [3]:
# Definir el dataset
bank_account_fraud_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 32 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   fraud_bool                        1000000 non-null  int64  
 1   income                            1000000 non-null  float64
 2   name_email_similarity             1000000 non-null  float64
 3   prev_address_months_count         1000000 non-null  int64  
 4   current_address_months_count      1000000 non-null  int64  
 5   customer_age                      1000000 non-null  int64  
 6   days_since_request                1000000 non-null  float64
 7   intended_balcon_amount            1000000 non-null  float64
 8   payment_type                      1000000 non-null  object 
 9   zip_count_4w                      1000000 non-null  int64  
 10  velocity_6h                       1000000 non-null  float64
 11  velocity_24h                      1000

### **Descripción del dataset**
#### ***Forma del dataset***
El dataset "bank account fraud" es de origen sintético que consta de 1 Millón de Filas y 32 Columnas.

#### ***Descripción de las variables***
- ***fraud_bool (binaria):*** tipo de dato integer, indica si la solicitud es fraudulenta o no. 
- ***income (numérica):*** tipo float, ingreso anual del solicitante, en deciles. Rango entre 0.1 y 0.9.
- ***name_email_similarity (numérica):*** tipo de dato float, medida de similaridad entre el email del solicitante y su nombre.
- ***prev_address_months_count (numérica):*** tipo de dato integer, cantidad de meses en la que el solicitante registro una dirección anterior. Ejemplo: si aplica, la dirección previa del solicitante. Rango entre -1 y 380 meses, donde -1 es un valor que falta.   
- ***current_address_months_count (numérica):*** tipo de dato integer, cantidad de meses que el applicante se mantiene en la misma dirección. Rangos entre -1 y 429 meses, donde -1 es un valor que falta. 
- ***customer_age (numérica):*** tipo de dato integer, edad del solicitante en años, redondeado a la decada. Rangos de 10 a 90 años.
- ***days_since_request (númerica):*** tipo de dato float, número de días que han transcurrido desde que la applicación fue realizada.
- ***intended_balcon_amount (numérica):*** tipo de dato float, cantidad inicial transferida por el solicitante. Rangos entre -16 y 114, los valores negativos indican valores que faltan.
- ***payment_type (categórica):*** tipo de dato object, tipo de plan de pagos de creditos. Cinco posibles valores (Anónimos)
- ***zip_count_4w (numérica):*** tipo de dato integer, número de solicitantes dentro del mismo código postal en las últimas cuatro semanas. Rangos entre 1 y 68030.
- ***velocity_6h (numérica):*** tipo de dato float, velocidad del total de solicitudes realizadas en las últimas seis horas. Ejemplo: promedio de solicitudes realizadas por hora en las últimas 6 horas. Rango entre -175 y 16818.
- ***velocity_24h (numérica):*** tipo de dato float, velocidad del total de solicitudes realizadas en las últimas 24 horas. Ejemplo: promedio de solicitudes realizadas por hora en las últimas 24 horas. Rango entre 1297 y 9586.
- ***velocity_4w (numérica):*** tipo de dato float, velocidad del total de solicitudes realizadas en las últimas 4 semanas. Ejemplo: promedio de solicitudes realizadas por hora en las últimas 4 semanas. Rango entre 2825 y 7020.
- ***bank_branch_count_8w (numérica):*** tipo de dato integer, número de solicitudes totales en la sucursal del banco seleccionada en las últimas 8 semanas. Rangos entre 0 y 2404.
- ***date_of_birth_distinct_emails_4w (numérica):*** tipo de dato integer, número de email por solicitantes con la misma fecha de nacimiento en las últimas 4 semanas. Rango entre 0 y 39. 
- ***employment_status (categórica):*** tipo de data object, situación laboral del solicitante. Siete posibles valores (anónimos).
- ***credit_risk_score (numérica):*** tipo de dato integer, puntaje inteno de riesgo de solicitud. Rangos entre -191 y 389.
- ***email_is_free (binaria):*** tipo de dato integer, dominio del email de la solicitud, puede ser gratis o pago.
- ***housing_status (categórica):*** tipo de dato object, situación residencial actual del solicitante. Siete valores posibles (anonimo).
- ***phone_home_valid (binaria):*** tipo de dato integer, validez del número de télefono proporcionado.
- ***phone_mobile_valid (binaria):*** tipo de dato integer, validez del número de télefono movil proporcionado.
- ***bank_months_count (numérica):*** tipo de dato integer, antigüedad de cuenta anterior en meses, si la tiene.
- ***has_other_cards (binaria):*** tipo de dato integer, si el solicitante tiene otras tarjetas del mismo banco.
- ***proposed_credit_limit (numérica):*** tipo de dato float, límite de crédito propuesto por al solicitante. Rango entre 200 y 2000.
- ***foreign_request (binaria):*** tipo de dato integer, si el país de origen del requerimiento es diferente al país del banco.
- ***source (categórica):*** tipo de dato object, medio online para la solicitud. Puede ser navegador (INTERNET) o app (TELEAPP)
- ***session_length_in_minutes (numérica):*** tipo de dato float, duración de la sesión del usuario en la página web del usuario en minutos. Rangos entre -1 y 107, donde -1 representa un valor faltante. 
- ***device_os (categórica):*** tipo de dato object, sistema operativo del dispositivo donde se realiza el requerimiento. Los posibles valores son: Windows, macOS, Linux, X11 u otro.
- ***keep_alive_session (binaria):*** tipo de dato integer, opción del usuario al cerrar sesión.
- ***device_distinct_emails_8w (numérica):*** tipo de dato integer, número de emails distintos en la página web del banco desde el dispositivo usadoen las últimas ocho semanas. Rango entre -1 y 2 emails, donde -1 es un valor faltante.
- ***device_fraud_count (numérica):*** tipo de dato integer, número de solicitudes fraudulentas con el dispositivo usado. Rango entre 0 y 1.
- ***month (numérica):*** tipo de dato integer, mes cuando la solicitud fue hecha. Rango entre 0 y 7.

#### **Acciones**
- Conocer el porcentaje de valores faltantes en las columnas que correspondan:

    [prev_address_months_count, intended_balcon_amount, session_length_in_minutes, device_distinct_emails_8w]

- Cambiar los tipos de datos donde correspondan:

    [proposed_credit_limit] ==> float a integer

    [source] ==> INTERNET -> browser | TELEAPP -> app

In [4]:
import numpy as np

# Convertir los valores negativos en valores faltantes
# Definir las columnas a convertir en NaN
columns_to_check = ['prev_address_months_count', 'intended_balcon_amount', 'session_length_in_minutes', 'device_distinct_emails_8w']

# Remplazar los valores negativos con NaN en las columnas específicadas
bank_account_fraud_df[columns_to_check] = bank_account_fraud_df[columns_to_check].applymap(lambda x: np.nan if x < 0 else x)

bank_account_fraud_df[columns_to_check].info()

C:\Users\fevc_\AppData\Local\Temp\ipykernel_15280\2457823895.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bank_account_fraud_df[columns_to_check] = bank_account_fraud_df[columns_to_check].applymap(lambda x: np.nan if x < 0 else x)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   prev_address_months_count  287080 non-null  float64
 1   intended_balcon_amount     257477 non-null  float64
 2   session_length_in_minutes  997985 non-null  float64
 3   device_distinct_emails_8w  999641 non-null  float64
dtypes: float64(4)
memory usage: 30.5 MB


### **Descripción de los valores faltantes**
- ***prev_address_months_count:*** 71.3% de los datos son NaN
- ***intended_balcon_amount:*** 74.2% de los datos son NaN
- ***session_length_in_minutes:*** 0.2% de los datos son NaN
- ***device_distinct_emails_8w:*** 0.03% de los datos son NaN

### **Acciones**
- Eliminar las columnas [prev_address_months_count, intended_balcon_amount] debido al alto porcentaje de valores faltantes, ya que interferirian con el análisis de los datos.
- Rellenar los valores faltantes de las columnas [session_length_in_minutes, device_distinct_emails_8w], ya que es insignificativa la cantidad de NaNs en estas columnas y podrían ser relevantes para el análisis de datos.